In [1]:
from sklearn.model_selection import KFold
#from mpl_toolkits import mplot_3d
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

np.random.seed(90)

In [8]:
class Layer:
    def __init__(self, n_input, n_neurons, activation=None, weights=None, bias=None):
        self.weights = weights if weights is not None else np.random.rand(n_input, n_neurons)
        self.activation = activation
        self.bias = bias if bias is not None else np.random.rand(n_neurons)
        self.last_activation = None
        self.error = None
        self.delta = None
        
    def activate(self, x):
        r = np.dot(x, self.weights) + self.bias
        self.last_activation = self._apply_activation(r)
        return self.last_activation
    
    def _apply_activation(self, r):
        if self.activation is None: return r
        if self.activation == 'tanh': return np.tanh(r)
        if self.activation == 'sigmoid': return 1/(1 + np.exp(-r))
        if self.activation == 'ReLU': return np.maximum(0, r)
        return r
    
    def apply_activation_derivative(self, r):
        if self.activation is None: return r
        if self.activation == 'tanh': return 1 - r ** 2
        if self.activation == 'sigmoid': return r * (1 - r)
        if self.activation == 'ReLU': return (r > 0).astype(int)

In [18]:
class NeuralNetwork:
    def __init__(self):
        self._layers = []
    
    def add_layer(self, layer):
        self._layers.append(layer)
        
    def feed_forward(self, X):
        for layer in self._layers:
            X = layer.activate(X)
        
        return X
    
    def back_propagation(self, X, y, learning_rate):
        output = self.feed_forward(X)
        
        for i in reversed(range(len(self._layers))):
            layer = self._layers[i]
            
            if layer == self._layers[-1]:
                layer.error = y - output
                layer.delta = layer.error * layer.apply_activation_derivative(output)
            else:
                next_layer = self._layers[i + 1]
                layer.error = np.dot(next_layer.weights, next_layer.delta)
                layer.delta = layer.error * layer.apply_activation_derivative(layer.last_activation)
                
        for i in range(len(self._layers)):
            layer = self._layers[i]
            input_to_use = np.atleast_2d(X if i == 0 else self._layers[i - 1].last_activation)
            layer.weights += layer.delta * input_to_use.T * learning_rate
            
    def train(self, X, y, learning_rate, max_epochs):
        kf = KFold(n_splits=10)
        kf.get_n_splits(X)
        mse = np.mean(np.square(y - nn.feed_forward(X)))
        for i in range(max_epochs):
            for train_index, test_index in kf.split(X):
                for j in train_index:
                    self.back_propagation(X[j], y[j], learning_rate)

In [22]:
X = np.genfromtxt('XNormalized.csv', delimiter=',', dtype=float)
y = np.genfromtxt('yNormalized.csv', delimiter=',', dtype=float)

nn = NeuralNetwork()
nn.add_layer(Layer(194,100, 'sigmoid'))
nn.add_layer(Layer(100,50, 'sigmoid'))
nn.add_layer(Layer(50,1, 'sigmoid'))
nn.train(X, y, 0.3, 100)

array([[5.000e+00, 3.000e+00, 4.000e+00, ..., 4.000e+00, 4.000e+00,
        4.000e+00],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 2.000e+00, 2.000e+00,
        2.000e+00],
       [6.480e+02, 2.900e+01, 2.388e+03, ..., 9.050e+02, 7.530e+02,
        3.910e+02],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 2.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 2.000e+00, 3.000e+00, ..., 0.000e+00, 7.000e+00,
        0.000e+00]])